In [ ]:
import pandas as pd

quick = True

if quick:
    start_year, end_year, chk, sampled, filename = [2005, 2021, False, True, 'df-light.pkl']
else:
    start_year, end_year, chk, sampled, filename = [2005, 2021, True, False, 'df-full.pkl']

from sklearn.model_selection import train_test_split

df = pd.read_pickle(f'./{filename}')
data = df.iloc[:, 1:]
target = df['grav']

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)
# X_train, y_train, X_test, y_test = pt.train_test_split_along_time(data, target, 2018)

In [ ]:
import time
from my_libs.encoder_custom import EncoderCustom

start_time = time.time()

# cols_target_encoded = ['dep', 'age']
cols_target_encoded = []
cols_onehot_encoded = X_train.columns.drop(cols_target_encoded)

encoder = EncoderCustom(cols_target_encoded=cols_target_encoded, cols_onehot_encoded=cols_onehot_encoded)

X_train_rs, y_train_rs = encoder.transform(X_train, y_train, 'Train')
X_test_rs,  y_test_rs  = encoder.transform(X_test,  y_test,  'Test')

print("--- Features encoding performed in %s seconds ---" % (time.time() - start_time))

In [ ]:
from my_libs.model_evaluator import ModelEvaluator

params_dt = {'max_depth': 15, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_split': 3}
model_type = 'DecisionTreeClassifier'

evaluator = ModelEvaluator(model_type=model_type, params=params_dt, X_train=X_train_rs, y_train=y_train_rs, X_test=X_test_rs, y_test=y_test_rs)
evaluator.evaluate()

In [ ]:
from my_libs.model_evaluator import ModelEvaluator

# params_rf_old = {'n_estimators': 34, 'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2}
# params_rf = {'n_estimators': 57, 'criterion': 'entropy', 'max_depth': 39, 'min_samples_split': 4}
params_rf = {'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 50}
model_type = 'RandomForestClassifier'

evaluator = ModelEvaluator(model_type=model_type, params=params_rf, X_train=X_train_rs, y_train=y_train_rs, X_test=X_test_rs, y_test=y_test_rs)
evaluator.evaluate()